In [79]:
### pyactup implementation 
### Author: CherYang
### Date: 2020-02-14
##############################################################

## init
import pyactup
import pyibl
import pprint
import numpy
import random
import math
import pprint as p
import matplotlib.pyplot as plt

### An abstract reward task stimulus: house stimulus amount of money associated with it
class HouseStimulus:
    def __init__(self, stimulus, amount):
        self.stimulus=stimulus
        self.amount=amount
        ### decide category
        self.category=self.decide_category(self.amount) 
    
    def decide_category(self, amount):
        if amount >=2.5: 
            return 'high'
        else:
            return 'low'
    def __str__(self):
        return (self.stimulus, self.amount, self.category)

class Task:
    def __init__(self, num_houses):
        self.n=num_houses
        self.reward_task_stimuli=self.create_house_stimuli(num_houses)
        self.decision_task_stimuli=self.create_decision_task_stimuli(num_houses)
    
    #    """this function create a set of stimuli (n) as a list"""
    def create_house_stimuli(self, n, show=False):
        stimuli=[]
        for i in range(n):
            stimuli.append(HouseStimulus("house"+str(i+1), float(i)/2))
        ## test
        if show:
            print('--- reward task stimuli ---')
            for i in stimuli:
                print (i.__str__())
        return stimuli   
    
    # this function generate 
    def create_decision_task_stimuli(self, n, show=False):
        stimuli=[]+self.create_house_stimuli(n)
        for i in range(n):
            stimuli.append(HouseStimulus("new_house"+str(i+1), float(i)/2))
        if show:
            print('--- decision-task stimuli ---')
            for i in stimuli:
                print (i.__str__())
        return stimuli 
    
    def reward_task(self, reward_task_stimuli):
        m=pyactup.Memory()
        m.reset()
        for h in reward_task_stimuli:
            m.learn(stimulus=h.stimulus, amount=h.amount, category=h.category)
            m.advance()
        return m

    def decision_task_single(self, m, decision_task_stimulus, show=False):
        retr_info=self.memory_task_single(m, decision_task_stimulus)
        choice="house0"
        if retr_info is not None:
            ### if retrieve the exact amount, and infer it is from high category
            ### choose this house
            if retr_info[2]=="high":
                choice=retr_info[0]
        if show:
            print("dt retr_info: ", retr_info)
            print("dt choice: ", choice)
        return choice
    
    def memory_task_single(self, m, memory_task_stimulus, show=False):
        c=m.retrieve(stimulus=memory_task_stimulus.stimulus, 
                   amount=memory_task_stimulus.amount, 
                   category=memory_task_stimulus.category)
        if show:
            print("mt: ",self.get_chunk_content(c))
        return self.get_chunk_content(c)
        
    
    #"""this function print items in memory (dict)"""
    def show_memory(m):
        print('--- memory ---')
        for k,v in m.items():
            print('key: ', k, 'v: ', v)

    #"""this function decompose the chunk into its content"""
    def get_chunk_content(self, c):
        if c is not None:
            #['house0', 2.61, 'unknown']
            house_name=list(c.values())[0]
            house_amount=list(c.values())[1]
            house_category=list(c.values())[2]
            return (house_name, house_amount, house_category)
        else: 
            return None

# plot the activation trace given the dict index  
def plot_each(index):
    plt.plot(base_act[index], 'black')
    plt.plot(act[index], 'o')
    
    plt.ylim([numpy.min(act[index]), numpy.max(act[index])])
    plt.title(f"retrieve high category 1000 times \nσ={m.noise}, d={m.decay}")
    plt.ylabel("activation")
    plt.xlabel("time retrieval")
    plt.show()


In [82]:
random.seed(0)
t=Task(10)
m=pyactup.Memory()

# reward task
m=t.reward_task(t.reward_task_stimuli)

# decision-task & memory task
counts = {"house"+str(k):0 for k in range(11)}
for trial in t.decision_task_stimuli:
    choice=t.decision_task_single(m,trial)
    counts[choice]+=1
pprint.pprint(counts)

{'house0': 15,
 'house1': 0,
 'house10': 1,
 'house2': 0,
 'house3': 0,
 'house4': 0,
 'house5': 0,
 'house6': 1,
 'house7': 1,
 'house8': 1,
 'house9': 1}
